In [64]:
import numpy as np
import torch
import torch.nn as nn
from dataset import CityscapesDataset
from torchvision.datasets import Cityscapes
from unet import *
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os 


In [65]:
# ALL CREDITS TO https://github.com/goldbattle/pytorch_unet/blob/master/script_train.py FOR DATASET CLASS!
img_data = CityscapesDataset("/home/charles/Desktop/cityscapes/", split='train', mode='fine')
img_batch = torch.utils.data.DataLoader(img_data, batch_size=1, shuffle=True)

In [66]:
# Hyperparameters and stuff
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 10
learning_rate = 0.001
epochs = 10
Net = build_unet()
Net.to(device)

# Since our goal here is to segment, we will use the cross entropy loss function and 4 classes (background, road, sky, car)
criterion = nn.CrossEntropyLoss() # for segment loss. could also be reconstruction loss, then it is N1 loss 
num_classes = img_data.num_classes
print(num_classes)
optimizer = torch.optim.Adam(Net.parameters(), lr=learning_rate)

cuda:0
4


In [67]:
for epoch in range(100):
    for idx_batch, (imagergb, labelmask, labelrgb) in enumerate(img_batch):
        imagergb = imagergb.to(device)
        labelmask = labelmask.to(device)
        labelrgb = labelrgb.to(device)
        # print number of input channels
        print(imagergb.shape)
        optimizer.zero_grad()
        output = Net(imagergb)
        loss = criterion(output, labelmask)
        loss.backward()
        optimizer.step()
        if idx_batch % 100 == 0:
            print("Epoch: {}/{}, Batch: {}/{}, Loss: {:.4f}".format(epoch+1, epochs, idx_batch+1, len(img_batch), loss.item()))

torch.Size([1, 3, 128, 256])


RuntimeError: Given groups=1, weight of size [32, 4, 3, 3], expected input[1, 3, 128, 256] to have 4 channels, but got 3 channels instead